# DX 704 Week 10 Project

In this project, you will implement document search within a question and answer database and assess its performance.


The full project description and a template notebook are available on GitHub: [Project 10 Materials](https://github.com/bu-cds-dx704/dx704-project-10).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download the SQuAD-explorer Data Set

You may use the code provided below.

In [22]:
!git clone https://github.com/rajpurkar/SQuAD-explorer

fatal: destination path 'SQuAD-explorer' already exists and is not an empty directory.


In [23]:
import json

In [24]:
with open("SQuAD-explorer/dataset/train-v1.1.json") as fp:
    train_data = json.load(fp)

In [25]:
type(train_data)

dict

In [26]:
list(train_data.keys())

['data', 'version']

In [27]:
type(train_data["data"])

list

In [28]:
len(train_data["data"])

442

In [29]:
type(train_data["data"][0])

dict

In [30]:
train_data["data"][0].keys()

dict_keys(['title', 'paragraphs'])

In [31]:
train_data["data"][0]["title"]

'University_of_Notre_Dame'

In [32]:
len(train_data["data"][0]["paragraphs"])

55

In [33]:
train_data["data"][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [34]:
sum(len(doc["paragraphs"]) for doc in train_data["data"])

18896

## Part 2: Restructure JSON Data for Processing

Parse the file "SQuAD-explorer/dataset/train-v1.1.json" above to produce a file "parsed.tsv" with columns document_title, paragraph_index, and paragraph_context.
The paragraph_index column should be zero-indexed, so zero for the first paragraph of each document.
Use pandas `to_csv` method to write the file since there are many quotes and other issues to handle otherwise.

In [35]:
# YOUR CHANGES HERE

import json
import pandas as pd

with open("SQuAD-explorer/dataset/train-v1.1.json", "r", encoding="utf-8") as f:
    squad_data = json.load(f)

rows = []
for doc in squad_data["data"]:
    title = doc.get("title", "")
    for idx, paragraph in enumerate(doc.get("paragraphs", [])):
        context = paragraph.get("context", "")
        rows.append({
            "document_title": title,
            "paragraph_index": idx,
            "paragraph_context": context
        })

df = pd.DataFrame(rows, columns=["document_title", "paragraph_index", "paragraph_context"])

df.to_csv("parsed.tsv", sep="\t", index=False)


Submit "parsed.tsv" in Gradescope.

## Part 3: Prepare Suitable Paragraph Vectors for Document Search

Design and implement paragraph vectors based on their text with length 1024.
Note that this will be much smaller than the number of distinct words in the training data.

Hint: you can base your vectors on any techniques covered in this module so far.
Beware that they will be automatically assessed (along with the question vectors of part 4) to make sure they retain useful information.

In [37]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
import json

df = pd.read_csv("parsed.tsv", sep="\t")

vectorizer = TfidfVectorizer(
    max_features=10000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2
)

tfidf_matrix = vectorizer.fit_transform(df["paragraph_context"])
svd = TruncatedSVD(n_components=1024, random_state=42)
paragraph_vectors = svd.fit_transform(tfidf_matrix)

paragraph_vectors = normalize(paragraph_vectors, norm="l2")

paragraph_vectors = np.round(paragraph_vectors, 5)

df_out = df[["document_title", "paragraph_index"]].copy()
df_out["paragraph_vector_json"] = [json.dumps(vec.tolist()) for vec in paragraph_vectors]
df_out.to_csv("paragraph-vectors.tsv.gz", sep="\t", index=False, compression="infer")


Save your paragraph vectors in a file "paragraph-vectors.tsv.gz" with columns document_title, paragraph_index, and paragraph_vector_json where paragraph_vector_json is a JSON encoded list.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [16]:
# YOUR CHANGES HERE

metadata = pd.read_csv("paragraph_metadata.tsv", sep="\t")

paragraph_vectors = np.load("paragraph_vectors.npy")

print(f"Loaded {len(metadata)} metadata rows and {len(paragraph_vectors)} vectors")

metadata["paragraph_vector_json"] = [
    json.dumps(vec.tolist()) for vec in paragraph_vectors
]

output_path = "paragraph-vectors.tsv.gz"
metadata.to_csv(output_path, sep="\t", index=False, compression="infer")


Loaded 18896 metadata rows and 18896 vectors


Submit "paragraph-vectors.tsv.gz" in Gradescope.

## Part 4: Encode Question Vectors with the Same Design

Read the questions in "questions.tsv" and encode them in the same way that you encoded the paragraph vectors.

In [17]:
import pandas as pd
import numpy as np
import json
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

df = pd.read_csv("parsed.tsv", sep="\t")
vectorizer = TfidfVectorizer(max_features=10000, stop_words="english", ngram_range=(1, 2), min_df=2)
tfidf_matrix = vectorizer.fit_transform(df["paragraph_context"])
svd = TruncatedSVD(n_components=1024, random_state=42)
paragraph_vectors = svd.fit_transform(tfidf_matrix)
paragraph_vectors = normalize(paragraph_vectors, norm="l2")
joblib.dump(vectorizer, "tfidf_vectorizer.joblib")
joblib.dump(svd, "svd_model.joblib")
df_out = df[["document_title", "paragraph_index"]].copy()
df_out["paragraph_vector_json"] = [json.dumps(vec.tolist()) for vec in paragraph_vectors]
df_out.to_csv("paragraph-vectors.tsv.gz", sep="\t", index=False, compression="infer")

vectorizer = joblib.load("tfidf_vectorizer.joblib")
svd = joblib.load("svd_model.joblib")
questions_df = pd.read_csv("questions.tsv", sep="\t")
tfidf_q = vectorizer.transform(questions_df["question"])
question_vectors = svd.transform(tfidf_q)
question_vectors = normalize(question_vectors, norm="l2")
questions_df["question_vector_json"] = [json.dumps(vec.tolist()) for vec in question_vectors]
questions_df.to_csv("question-vectors.tsv.gz", sep="\t", index=False, compression="infer")



Save your question vectors in "question-vectors.tsv" with columns question_id and question_vector_json.

In [18]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np
import json
import joblib
from sklearn.preprocessing import normalize

vectorizer = joblib.load("tfidf_vectorizer.joblib")
svd = joblib.load("svd_model.joblib")

questions_df = pd.read_csv("questions.tsv", sep="\t")

tfidf_q = vectorizer.transform(questions_df["question"])
question_vectors = svd.transform(tfidf_q)
question_vectors = normalize(question_vectors, norm="l2")

questions_out = pd.DataFrame({
    "question_id": questions_df["question_id"],
    "question_vector_json": [json.dumps(vec.tolist()) for vec in question_vectors]
})

questions_out.to_csv("question-vectors.tsv.gz", sep="\t", index=False, compression="infer")


Submit "question-vectors.tsv" in Gradescope.

## Part 5: Match Questions to Paragraphs using Nearest Neighbors

Match your question vectors to paragraph vectors and identify the top 5 paragraph vectors for each question using nearest neighbors.
Specifically, use the Euclidean distance between the vectors.


In [38]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import joblib

df_paragraph = pd.read_csv("paragraph-vectors.tsv.gz", sep="\t", compression="infer")
paragraph_vectors = np.array([json.loads(x) for x in df_paragraph["paragraph_vector_json"]], dtype=np.float32)
paragraph_vectors = np.round(paragraph_vectors, 5)

df_question = pd.read_csv("question-vectors.tsv", sep="\t")
question_vectors = np.array([json.loads(x) for x in df_question["question_vector_json"]], dtype=np.float32)
question_vectors = np.round(question_vectors, 5)


distances = pairwise_distances(question_vectors, paragraph_vectors, metric='euclidean')


top_k = 5
top_indices = np.argsort(distances, axis=1)[:, :top_k]

matches = []
for q_idx, paragraph_idxs in enumerate(top_indices):
    q_id = df_question.loc[q_idx, "question_id"]
    for rank, p_idx in enumerate(paragraph_idxs, start=1):
        matches.append({
            "question_id": q_id,
            "question_rank": rank,
            "document_title": df_paragraph.loc[p_idx, "document_title"],
            "paragraph_index": df_paragraph.loc[p_idx, "paragraph_index"]
        })


df_matches = pd.DataFrame(matches)
df_matches.to_csv("question-matches.tsv", sep="\t", index=False)


Save your top matches in a file "question-matches.tsv" with columns question_id, question_rank, document_title, and paragraph_index.


In [20]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
import json
from sklearn.metrics import pairwise_distances

paragraph_df = pd.read_csv("paragraph-vectors.tsv.gz", sep="\t")
paragraph_vectors = np.array([json.loads(x) for x in paragraph_df["paragraph_vector_json"]], dtype=np.float32)

question_df = pd.read_csv("question-vectors.tsv", sep="\t")
question_vectors = np.array([json.loads(x) for x in question_df["question_vector_json"]], dtype=np.float32)

distances = pairwise_distances(question_vectors, paragraph_vectors, metric='euclidean')

top_k = 5
top_indices = np.argsort(distances, axis=1)[:, :top_k]

matches = []
for q_idx, paragraph_idxs in enumerate(top_indices):
    q_id = question_df.loc[q_idx, "question_id"]
    for rank, p_idx in enumerate(paragraph_idxs, start=1):
        matches.append({
            "question_id": q_id,
            "question_rank": rank,
            "document_title": paragraph_df.loc[p_idx, "document_title"],
            "paragraph_index": paragraph_df.loc[p_idx, "paragraph_index"]
        })

matches_df = pd.DataFrame(matches)
matches_df.to_csv("question-matches.tsv", sep="\t", index=False)


Submit "question-matches.tsv" in Gradescope.

## Part 6: Spot Check Question and Paragraph Matches

Review the paragraphs matched to the first 5 questions (sorted by question_id ascending).
Which paragraph was the worst match for each question?


Submit "worst-paragraphs.tsv" in Gradescope.

Write a file "worst-paragraphs.tsv" with three columns question_id, document_title, paragraph_index.

In [21]:
import pandas as pd

matches_df = pd.read_csv("question-matches.tsv", sep="\t")

matches_df = matches_df.sort_values(by=["question_id", "question_rank"])

first5_questions = matches_df["question_id"].drop_duplicates().head(5)

worst_matches = matches_df[matches_df["question_id"].isin(first5_questions) & (matches_df["question_rank"] == 5)]

worst_matches = worst_matches[["question_id", "document_title", "paragraph_index"]]

worst_matches.to_csv("worst-paragraphs.tsv", sep="\t", index=False)



## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 8: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.